# Known Problem:
- **Unresolved**
- **Partially Resolved**
    1. Certain websites cause the error of, specified here: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests, which is a connection error. 
        - this appears to be windows specific as this was not a relevant issue when testing code in ubuntu 22.04, some forum reflect similar message
        - FAILED SOLUTION: first response in that stackoverflow page, still kept the stuff bc it seemed like better QOL
        - ATTEMPTED SOLUTION: used try and except. NOTE: this solution is not ideal as it does not resolve the issue, it just ignores it and moves on.
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify
        - ATTEMPTED SOLUTION: used try except to use verify=False when ssl error occurs. Nested try except to ensure that both connection error and ssl error are handled.

    

In [2]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import re

In [3]:
import ssl
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\William
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\William
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\William
[nltk_data]     Zhang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Step 1: Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 2

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

search_results = []
for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    # Make a GET request to retrieve the search result page
    response = session.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)

    # Step 2: Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)

            # Make a GET request to the URL of the search result
            try:
                try:
                    print("trying 1")
                    result_response = session.get(url, headers=headers)
                except:
                    try:
                        print("trying 2")
                        result_response = session.get(url, headers=headers, verify=False)
                    except:
                         print(1/0)
            except:
                print("Error: Website refuses connection.")
            # Extract visible text content from the HTML response
            result_soup = BeautifulSoup(result_response.content, "html.parser")
            text = result_soup.get_text()

            # Remove excess whitespace from the extracted text
            text = ' '.join(text.split())

            search_results.append({"Title": title, "URL": url, "Text": text})

trying 1
trying 1
trying 1
trying 1
trying 1
trying 2
Error: Website refuses connection.
trying 1
trying 1
trying 1
trying 1
trying 1
trying 1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


trying 1
trying 1
trying 1
trying 1
trying 1
trying 1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [5]:
df = pd.DataFrame(search_results)
print(len(df))
#unfiltered dataframe
df.head()

17


,Title,URL,Text
0,Taylor series,https://en.wikipedia.org/wiki/Taylor_series,Taylor series - Wikipedia Jump to content Main...
1,Taylor Series -- from Wolfram MathWorld,https://mathworld.wolfram.com/TaylorSeries.html,Taylor Series -- from Wolfram MathWorld TOPICS...
2,Taylor Series,https://www.mathsisfun.com/algebra/taylor-seri...,Taylor Series Advanced Show Ads Hide Ads About...
3,A Gentle Introduction to Taylor Series,https://machinelearningmastery.com/a-gentle-in...,A Gentle Introduction to Taylor Series - Machi...
4,Calculus II - Taylor Series,https://tutorial.math.lamar.edu/classes/calcii...,A Gentle Introduction to Taylor Series - Machi...


In [6]:
stop_words = stopwords.words("english") #english filler words

def preprocess(text):
    '''
    Input a text block and filter out unneeded characters
    returns a filtered text block in the form of a str
    Function filters whitespace, numbers, special characters, stopwords; handles case normalization 
    Function tokenize and lemmatize the input text
    '''
    # Remove special characters and numbers
    cleaned_text = re.sub(r"[^a-zA-Z]+", " ", text)
    
    # Convert to lowercase and split into words
    words = cleaned_text.lower().split()
    
    # Remove stop words and single-character words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    joined_filtered_words =  " ".join(filtered_words)
    
    #tokenize and lemmatize the words
    tokenized_text = word_tokenize(joined_filtered_words)
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_text = [wordnet_lemmatizer.lemmatize(word) for word in tokenized_text]
    
    return lemmatized_text


In [7]:
example = df["Text"][0]
processed_example = preprocess(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(processed_example))

length of unfiltered text:  51507
length of filtered text:  3714


In [8]:
#Properly using the apply() function onto a pandas column
#preprocess all data in dataframe

df["Text"] = df["Text"].apply(preprocess)

In [9]:
print(len(df["Text"][0]))

3714


In [10]:
#filtered text for first row entry
df["Text"][0][0:3001]

['taylor',
 'series',
 'wikipedia',
 'jump',
 'content',
 'main',
 'menu',
 'main',
 'menu',
 'move',
 'sidebar',
 'hide',
 'navigation',
 'main',
 'pagecontentscurrent',
 'eventsrandom',
 'articleabout',
 'wikipediacontact',
 'usdonate',
 'contribute',
 'helplearn',
 'editcommunity',
 'portalrecent',
 'changesupload',
 'file',
 'language',
 'language',
 'link',
 'top',
 'page',
 'across',
 'title',
 'search',
 'search',
 'create',
 'accountlog',
 'personal',
 'tool',
 'create',
 'account',
 'log',
 'page',
 'logged',
 'editor',
 'learn',
 'contributionstalk',
 'content',
 'move',
 'sidebar',
 'hide',
 'top',
 'definition',
 'example',
 'history',
 'analytic',
 'function',
 'approximation',
 'error',
 'convergence',
 'toggle',
 'approximation',
 'error',
 'convergence',
 'subsection',
 'generalization',
 'list',
 'maclaurin',
 'series',
 'common',
 'function',
 'toggle',
 'list',
 'maclaurin',
 'series',
 'common',
 'function',
 'subsection',
 'exponential',
 'function',
 'natural',
 '